In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import math
import time
import json
import mat73
import warnings
import numpy as np
from tqdm import trange
import matplotlib.pyplot as plt
from scipy.optimize import nnls
import matplotlib.image as mpimg
from sklearn.decomposition import NMF
from scipy.io import loadmat, savemat
from matplotlib.patches import Rectangle
from sklearn.decomposition import FastICA
from matplotlib.colors import ListedColormap
from utils import *

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
abs_coeff, skin_dict, water_dict = {}, {}, {}
wave_abs = np.load('../data/hbo2hbchpr_57.npy')
plt3d = loadmat('../data/3DPlot.mat')
X, Y = plt3d['x'], plt3d['y']
abs_dict = {}

for idx, wave in enumerate(np.arange(700, 981, 5)):
    abs_coeff[wave] = wave_abs[idx]
    skin = (0.244 + 85.3 * math.exp(-(wave - 154)/66.2)) / 100
    melanin = (1.70 * 1e12 * math.pow(wave, -3.48)) / 100
    skin_dict[wave] = np.array([skin, melanin])
    abs_dict[int(wave)] = list(np.append(np.append(wave_abs[idx], skin), melanin))

wave_list = [750, 800, 850]
coeffs = np.vstack([abs_coeff[wave] for wave in wave_list])
fcoeffs = np.vstack([np.append(abs_coeff[wave], skin_dict[wave], axis = 0) for wave in wave_list])
norm_coeffs = np.array([fcoeffs.T[idx] / max(coeffs.T[idx]) for idx in range(coeffs.shape[1])]).T
legend = ['HbO', 'Hb', 'Cholesterol']
melorder = ['HbO2', 'Hb', 'Cholesterol', 'Melanin']
skinorder = ['HbO2', 'Hb', 'Cholesterol', 'Skin Baseline']
hbhbo2fat = np.copy(coeffs)[:, [0, 1, 2]]
hbhbo2fatskin = np.copy(fcoeffs[:, [0, 1, 2, 4]])
hbhbo2fatmelanin = np.copy(fcoeffs[:, [0, 1, 2, 5]])

mixed_coeffs = []
for idx in [760, 808, 915]:
    if idx == 808:
        ids = 0.69 * abs_coeff[805] + 0.31 * abs_coeff[940]
    else:
        ids = abs_coeff[idx]
    mixed_coeffs.append(ids)
mixed_coeffs = np.array(mixed_coeffs)[:, :3]

In [ ]:
with open("exps.json") as file:
    expdict = json.load(file)

# Hypercapnia

In [ ]:
dir = "/media/prameth/Inland SSD/AcousticX Data/data/beam-5/11012023/"
fdir = sorted([file for file in os.listdir(dir) if file.endswith(".mat")])
for idx in range(len(fdir)):
    print(idx, fdir[idx])

In [ ]:
fidx = 0
pafile = loadmat(os.path.join(dir, fdir[fidx]))['beamformed']
pafile = pafile.transpose((2, 0, 1))
paw1, paw2, frames = pafile[list(range(0, pafile.shape[0], 2))], pafile[list(range(1, pafile.shape[0], 2))], pafile.shape[0]
rois = textparse(os.path.join(dir, fdir[0].split(".")[0] + ".txt"))
wave_dict = {760: 0, 808: 1, 915: 2}
waves = [int(fdir[fidx].split("_")[1]), int(fdir[fidx].split("_")[3])]
suptitle = f"{dir.split('/')[-2]} {expdict[dir.split('/')[-2]]} {' '.join([str(waves[i] if waves[i]!=808 else '808/940') + 'NM' for i in range(len(waves))])}"
print(suptitle)

In [ ]:
nrois = round(len(rois) / 2)
ratios, imgfile = 2 * [(0, 0.2), (0.2, 0.8), (0.8, 1)], []
wave_list = [wave_dict[idx] for idx in waves]
swv = [str(wave) if wave != 808 else "808/940" for wave in waves]
title = [f'PRE CO2 {swv[0]}NM', f'CO2 {swv[0]}NM', f'POST CO2 {swv[0]}NM', f'PRE CO2 {swv[1]}NM', f'CO2 {swv[1]}NM', f'POST CO2 {swv[1]}NM']
for idx in range(len(ratios)):
    if idx < 3:
        imgfile.append(np.mean(paw1[round(ratios[idx][0] * frames / 2): round(ratios[idx][1] * frames / 2), rois[8][0]: rois[8][1], rois[8][2]: rois[8][3]], axis = 0))
    else:
        imgfile.append(np.mean(paw2[round(ratios[idx][0] * frames / 2): round(ratios[idx][1] * frames / 2), rois[8][0]: rois[8][1], rois[8][2]: rois[8][3]], axis = 0))
imgfile = np.array(imgfile)
plt.figure(figsize = (12, 7))
for idx in range(len(imgfile)):
    plt.subplot(2, 3, idx + 1)
    if idx < 3:
        plt.imshow(imgfile[idx], cmap = 'hot', extent = [0, 1, 0, 1])
        plt.clim([0, np.max(imgfile[:3])])
    else:
        plt.imshow(imgfile[idx], cmap = 'hot', extent = [0, 1, 0, 1])
        plt.clim([0, np.max(imgfile[3:])])
    plt.colorbar()
    plt.title(title[idx])
plt.suptitle(t = f"{suptitle} Multi-Wavelength")
plt.tight_layout()
plt.show()

In [ ]:
title_order = ['PRE CO2', 'CO2', 'POST CO2']
order = ['HbO', 'Hb', 'Cholesterol']
imgdict, didx = {}, 0
for i in [[0, 1], [0, 2], [1, 2]]:
    for idx, j in enumerate([[0, 3], [1, 4], [2, 5]]):
        imgdict[didx] = (title_order[idx], i, j)
        didx += 1
imgidx = 1
plt.figure(figsize = (24, 11))
for idx in range(9):
    exp_unmixed = run_linear_unmixing(normalize(imgfile[imgdict[idx][2], :, :].copy()), mixed_coeffs[wave_list][:, imgdict[idx][1]])
    for xidx in range(2):
        plt.subplot(3, 6, imgidx)
        plt.imshow(exp_unmixed[:, :, xidx], cmap = "hot", extent = [0, 1, 0, 1])
        plt.colorbar()
        plt.title(f"{imgdict[idx][0]} {order[imgdict[idx][1][xidx]]}")
        # plt.clim([0, np.max(exp_unmixed)])
        imgidx += 1
plt.suptitle(t = f"{suptitle} Linear Unmixing")
plt.tight_layout()
plt.show()

In [ ]:
unmixarr = run_linear_unmixing(normalize(imgfile[imgdict[3][2], :, :]).copy(), mixed_coeffs[wave_list][:, imgdict[3][1]])
plt.figure(figsize = (10, 5))
plt.subplot(1, 2, 1)
plt.imshow(unmixarr[:, :, 0], cmap = "hot", extent = [0, 1, 0, 1])
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(unmixarr[:, :, 1], cmap = "hot", extent = [0, 1, 0, 1])
plt.colorbar()

In [ ]:
plt.figure(figsize = (15, 4))
for idx in range(3):
    exp_unmixed = run_linear_unmixing(normalize(imgfile[imgdict[idx][2], :, :].copy()), mixed_coeffs[wave_list][:, imgdict[idx][1]])
    plt.subplot(1, 3, idx + 1)
    plt.imshow(exp_unmixed[:, :, 0] / (exp_unmixed[:, :, 0] + exp_unmixed[:, :, 1]), cmap = 'bwr', extent = [0, 1, 0, 1])
    plt.colorbar()
    plt.title(f"{imgdict[idx][0]}")
    plt.clim([0.6, 1])
plt.suptitle(f"{suptitle} SO$_{2}$ Distribution")
plt.tight_layout()
plt.show()

In [ ]:
means, mavgs, same_size, window_size = [], [], False, 20
plt.figure(figsize = (25, 8))
plt.subplot(2, 5, 1)
plt.imshow(mpimg.imread(os.path.join(dir, fdir[fidx].split(".")[0] + "-1.jpg"))[50:580, 100:700])
plt.axis('off')
plt.title(f"{swv[0]} ROIs")
plt.subplot(2, 5, 6)
plt.imshow(mpimg.imread(os.path.join(dir, fdir[fidx].split(".")[0] + "-2.jpg"))[50:580, 100:700])
plt.axis('off')
plt.title(f"{swv[1]} ROIs")
for idx, subidx in enumerate(list(set(list(range(10))) - set([0, 5]))):
    plt.subplot(2, 5, subidx + 1)
    if idx < nrois:
        mean = np.mean(paw1[:, rois[idx][0]:rois[idx][1], rois[idx][2]:rois[idx][3]], axis = (1, 2))
    else:
        mean = np.mean(paw2[:, rois[idx][0]:rois[idx][1], rois[idx][2]:rois[idx][3]], axis = (1, 2))
    mavg = moving_average_same(mean, window_size) if same_size else moving_average(mean, window_size)
    plt.plot(np.arange(len(mean[:-1])), mean[:-1], "b")
    plt.plot(np.arange((window_size / 2), (len(mavg[:-1]) + (window_size / 2))), mavg[:-1], 'r')
    means.append(mean)
    mavgs.append(mavg)
    plt.axvspan(round(frames * ratios[1][0] / 2), round(frames * ratios[1][1] / 2), color = 'coral', alpha = 0.4, lw = 0)
    plt.title(f'{waves[0] if idx < len(rois) else waves[1]} NM ROI - {(idx + 1) if idx < 4 else (idx - 3)}')
plt.suptitle(t = f"{suptitle} Time Series Intensity")
plt.tight_layout()
plt.show()

In [ ]:
indices, min_y = [4, 5, 7], 0.8
colors = ['b', 'g', 'r']
for j, idx in enumerate(indices):
  y = np.array(mavgs[idx][:-1])
  x = np.linspace(0, 2.5, len(y))
  y = y / max(y)
  plt.plot(x, y, colors[j], lw = 2.5)
  plt.text(-0.1, y[0], f'{j + 1}', fontsize = 20, fontweight = 'bold', color = colors[j])
  plt.axvspan(0.5, 2.0, color = 'lightskyblue', alpha = 0.3, lw = 0)
plt.xticks([0, 0.5, 2, 2.5], weight = 'bold', fontsize = 15)
plt.yticks([min_y, (1 + min_y)/2, 1], weight = 'bold', fontsize = 15)
plt.xlabel('Time (minutes)', fontsize = 18, fontweight = 'bold')
plt.ylabel('Norm PA Amp (a.u.)', fontweight = 'bold', fontsize = 18)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (25, 4))
plt.subplot(1, 5, 1)
plt.imshow(mpimg.imread(os.path.join(dir, fdir[fidx].split(".")[0] + "-3.jpg"))[50:580, 100:700])
plt.axis('off')
plt.title(f"{swv[0]} {swv[1]} SO2 ROIs")
for idx, ridx in enumerate(list(range(9, 13))):
    plt.subplot(1, 5, idx + 2)
    mean = np.mean(so2stack[:, rois[ridx][0]:rois[ridx][1], rois[ridx][2]:rois[ridx][3]], axis = (1, 2))
    mavg = moving_average(mean, 5)
    plt.plot(mean[:-1], "b")
    plt.plot(mavg[:-1], 'r')
    plt.axvspan(round(frames * ratios[1][0] / 2), round(frames * ratios[1][1] / 2), color = 'coral', alpha = 0.4, lw = 0)
    plt.title(f'ROI {idx + 1}')
plt.suptitle(t = f"{suptitle} SO$_{2}$ Time Series Analysis")
plt.tight_layout()
plt.show()

In [ ]:
indunmix = []
for idx in range(3):
    for cidx in range(3):
        indunmix.append(run_linear_unmixing(normalize(imgfile[imgdict[idx][2]].copy()), mixed_coeffs[wave_list][:, [cidx]])[:, :, 0])
indunmix = np.array(indunmix)

plt.figure(figsize = (12, 12))
for idx in range(9):
    plt.subplot(3, 3, idx + 1)
    plt.imshow(indunmix[idx], cmap = 'hot', extent = [0, 1, 0, 1])
    plt.colorbar()
    plt.title(f"{imgdict[idx][0]} {legend[idx % 3]}")
plt.tight_layout()
plt.show()